```{currentmodule} optimap
```

In [ ]:
from optimap.utils import jupyter_render_animation as render

```{tip}
Download this tutorial as a {download}`Jupyter notebook <converted/activation.ipynb>`, or a {download}`python script <converted/activation.py>` with code cells. We highly recommend using [Visual Studio Code](#vscode) to execute this tutorial.
```

# Tutorial 6: Activation Maps

This tutorial demonstrates how to compute and visualize activation maps from cardiac optical mapping data using ``optimap``. Activation maps display local activation times (LATs), which indicate when the tissue becomes electrically activated. These maps are crucial for understanding cardiac electrical activity and identifying abnormalities.

Computing local activation times corresponds to determining when the optical signal in a given pixel passes a certain pre-defined threshold or intensity value. For instance, if the optical trace is normalized and fluctuates between [0, 1] then the tissue could be defined as being 'electrically activated' when the time-series rises above or below 0.5 (depending on the fluorescent indicator and polarity of the signal).

Here, we will use an example data from {cite:t}`Rybashlykov2022` in which a planar action potential wave propagates across the ventricle of a mouse heart.

In [ ]:
import optimap as om
import monochrome as mc
import numpy as np
import matplotlib.pyplot as plt

filename = om.download_example_data("doi:10.5281/zenodo.5557829/mouse_41_120ms_control_iDS.mat")
video = om.load_video(filename)
metadata = om.load_metadata(filename)
print(f"Loaded video with shape {video.shape} and metadata {metadata}")
frequency = metadata["frequency"]

The `mouse_41_120ms_control_iDS.mat` file from the [Zenodo dataset](https://doi.org/10.5281/zenodo.5557829) shows a induced pacing beats in a mouse heart. The {func}`load_metadata` function loads the metadata from the MATLAB file, in this case the acquisition frame rate. We visualize the video using [Monochrome](https://github.com/sitic/Monochrome):

In [ ]:
# Show video
mc.show(video, name=filename.name, metadata=metadata)

In [ ]:
from IPython.display import Video
Video('https://cardiacvision.ucsf.edu/sites/g/files/tkssra6821/f/optimap-mouse_41_120ms_control_iDS_monochrome.mp4', html_attributes='controls autoplay loop width="100%"')

We use ``optimap``'s {func}`background_mask` function to create a mask of the heart:

In [ ]:
# remove background by masking
mask = om.background_mask(video[0], show=False)
mask = om.image.dilate_mask(mask, iterations=5, show=False)
om.image.show_mask(mask, video[0], title="Background mask");

Computing activation maps is highly noise-sensitive, so we need to need to apply strong filtering to the data. Here we use a spatio-temporal Gaussian filter and a spatial mean filter to smooth the data.

In [ ]:
video_filtered = om.video.smooth_spatiotemporal(video, sigma_temporal=1, sigma_spatial=1)
video_filtered = om.video.mean_filter(video_filtered, size_spatial=5)

# Normalize the video using a pixelwise sliding window
video_norm = om.video.normalize_pixelwise_slidingwindow(video_filtered, window_size=200)
video_norm[:, mask] = np.nan

Because the mouse heart was stained with the voltage-sensitive dye Di-4-ANEPPS, the tissue becomes darker when it depolarizes (negative signal / polarity):

In [ ]:
om.show_video_pair(video, video_norm, title1="original video",
                   title2="normalized video", interval=10)

# Or in Monochrome:
#
# mc.show(video, name="original video")
# mc.show(video_norm, name="normalized video")

In [ ]:
render(lambda: om.show_video_pair(video, video_norm, title1="original video", title2="normalized video", interval=20))

We can use the {func}`activation.find_activations` function to find pacing beats. Here we just average the signal over the whole heart to identify when the pacing beat occurs. The function will return a list of activation times in frames and plot detected activations with a red line:

In [ ]:
activations = om.activation.find_activations(1 - video_norm, fps=frequency)
print(f"Found {len(activations)} activation events at frames: {activations}")

In [ ]:
# Select a trace position close to the pacing site 
#
# traces, coords = om.select_traces(video_norm[:500], size=10, ref_frame=video[0])
# trace = om.extract_traces(video_norm, coords[0], size=10)
# activations = om.activation.find_activations(1 - trace)

# Here hardcoded position (141, 100) for demo purposes
fig, axs = plt.subplot_mosaic('ABB', figsize=(10, 2))
om.show_positions([(141, 100)], video[0], ax=axs['A'])
trace = om.extract_traces(video_norm, (141, 100), size=10)
activations = om.activation.find_activations(1 - trace, ax=axs['B'], fps=frequency)
plt.show()

print(f"Found {len(activations)} activation events at frames: {activations}")

Let's visualize some of the video frames as the wave propagates across the ventricles for the first pacing beat:

In [ ]:
figure, axs = plt.subplots(1, 7, figsize=(10, 3))
axs[0].imshow(video[0], cmap='gray')
axs[0].set_title('original')
axs[0].set_axis_off()

for i in range(1, 7):
    t = i * 2
    axs[i].imshow(video_norm[activations[0] + t], cmap='gray', vmin=0, vmax=1)
    axs[i].set_axis_off()
    time = t * (1000/frequency)  # convert to ms
    axs[i].set_title(f"{time:.1f} ms")
plt.axis('off')
plt.show()

We can now compute an activation map by identifying the local activation times in each pixel that correspond to when the action potential wave front passes through that pixel.

## Computing Activation Maps from Pixel-wise Normalized Optical Maps


A pixel-wise normalization was sufficient as opposed to a sliding-window pixel-wise normalization, see [Tutorial 2](signal_extraction.ipynb), because we isolated a short part of the video that is only 20 frames long. In other cases it might be necessary to use a sliding-window pixel-wise normalization or a frame-wise difference video (e.g. with motion), see below.

Let's plot some of the optical traces (manually selected so that they show locations which become subsequently activated):

In [ ]:
# Positions selected with the GUI:
# positions = om.select_positions(video[0])
positions =  [(134, 101), (14, 93), (94, 99), (53, 97)]

fig, axs = plt.subplots(1, 2, figsize=(10,5))
om.trace.show_positions(positions, video[0], ax=axs[0])
traces = om.extract_traces(video_norm,
                           positions,
                           size=10,
                           fps=frequency,
                           ax=axs[1])
axs[1].axhline(y=0.5, color='r', linestyle='dashed', label='threshold')
axs[1].text(0.03, 0.52, 'threshold', color='r')
plt.xlim(0, 0.12)
plt.show()

We can use ``optimap``'s {func}`compute_activation_map` function to automatically compute a two-dimensional activation map which shows the local activation times in every pixel:

In [ ]:
trace = om.extract_traces(video_norm, (149, 115), size=10)
activations = om.activation.find_activations(1 - trace, show=False)

idx = activations[2]
activation_map = om.compute_activation_map(video_norm[idx-3:idx + 16], inverted=True, fps=frequency, vmax=15, interpolate=False, show_contours=True, min_duration=0)

# mc.show(activation_map, cmap='turbo')

In [ ]:
# Refined mask manually with the GUI:
#
# mask = om.interactive_mask(image=video[0], initial_mask=mask)
# om.save_mask('mouse_41_120ms_control_iDS_mask.png', mask)

# Loading the mask from the file for demo purposes
mask_filename = om.download_example_data('mouse_41_120ms_control_iDS_mask.png')
mask = om.load_mask(mask_filename)
video_norm[:, mask] = np.nan

By default, {func}`compute_activation_map` and {func}`find_activation` calculates the local activation time (LAT) for each pixel as the time when the signal crosses a threshold of 0.5:

$$\text{LAT} = \text{argmin}_{t} \left( \text{signal}_{t+1} > 0.5 \right)$$

The plot (left panel) below demonstrates this behavior, the vertical lines are the detected local activation times.

If `invert=True` is set for {func}`find_activation`, the LAT is calculated as the time when the signal crosses a threshold of 0.5 in the opposite direction.

When `interpolation=True` is set, the LAT is calculated as the fractional time between the two frames that cross the threshold using linear interpolation (right panel):

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
ax=axs[0]
colors = ['blue', 'orange', 'green', 'red']
for ax, interpolate in zip(axs, [False, True]):
    om.show_traces(traces, ax=ax, colors=colors, linestyle='solid', marker='.')
    activations = om.activation.find_activations(1 - traces, interpolate=interpolate, show=False)
    for i in range(len(activations)):
        for activation in activations[i]:
            if 12 <= activation <= 28:
                ax.axvline(activation, linestyle='--', color=colors[i], alpha=0.6)
                ax.text(activation, 0.99, f'{activation:.1f}', 
                        rotation=90, va='top', ha='right', color=colors[i], fontsize=10)
    ax.axhline(y=0.5, color='r', linestyle='dashed', label='threshold')
    ax.text(25, 0.51, 'threshold', color='r')
    ax.set_xlim(12, 28)
    ax.grid()
    ax.set_title(f"interpolate=True" if interpolate else "interpolate=False (default)")
fig.suptitle("Non-interpolated vs interpolated LATs")
plt.show()

Here is the activation map of the last example when using linearly interpolated LATs:

In [ ]:
activation_map = om.compute_activation_map(video_norm[idx-3:idx + 16], inverted=True, interpolate=True, show_contours=True, fps=frequency, vmax=14)

## Adding Contour Lines to Activation Maps

Contour lines are a powerful visualization tool that can help highlight the wavefront propagation. They connect points with the same activation time, making it easier to visualize the speed and direction of propagation. Let's add contour lines to our activation map:

In [ ]:
levels = [3, 6, 9, 12, 15]
# video_norm2 = om.video.mean_filter(video_norm, size_spatial=5)
activation_map = om.compute_activation_map(video_norm[idx - 3:idx + 15], inverted=True, fps=frequency, vmax=15, show_contours=True, contour_levels=levels)

You can also combine contour lines with a raw image to visualize the propagation path over the heart tissue:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
om.show_image(video[idx], ax=ax)
# Add a red boundary around the mask
mask_boundary = ax.contour(~mask, levels=[0], colors='black', linewidths=1.5, alpha=0.8)
# Show contours
om.show_activation_map(
    activation_map,
    ax=ax,
    show_map=False,
    show_contours=True,
    contour_levels=range(2, 20, 2),
    contour_fontsize=10,
    contour_fmt='%1.0f ms',
    contour_args={'linewidths': 1.5, 'alpha': 0.8, 'cmap': 'turbo', 'colors': None})
# contour = ax.contour(activation_map, levels=contour_levels, cmap='turbo', linewidths=1.5, alpha=0.8)
# ax.clabel(contour, fontsize=12, fmt='%1.0f ms')
plt.tight_layout()
plt.show()

Note that we used the argument `inverted=True` due to the negative polarity of the signal ($- \Delta F / F$). If me had manually inverted the video beforehand or with calcium imaging data this would not be necessary. The range of local activation times can be displayed with:

In [ ]:
om.print_properties(activation_map)

In this case, the local activation times are given in milliseconds (based on argument `fps`) and they range between 0ms and 18.4ms. The function {func}`compute_activation_map` uses {func}`show_activation_map` to plot the activation map (which can be disabled with argument `show=False`):

In [ ]:
om.show_activation_map(activation_map, cmap="jet", title='Activation Map', show_colorbar=True, colorbar_title='Activation Time [ms]', vmax=14);

We have plotted the activation map using the `jet` colormap, here are some other options:

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(8, 3))
om.show_activation_map(activation_map, cmap='jet', show_colorbar=True, title='cmap=jet', ax=axs[0], colorbar_title=None)
om.show_activation_map(activation_map, cmap='magma', show_colorbar=True, title='cmap=magma', ax=axs[1], colorbar_title=None)
om.show_activation_map(activation_map, cmap='twilight_shifted', show_colorbar=True, title='cmap=twilight_shifted', ax=axs[2], colorbar_title=None)
plt.suptitle('Activation maps with different colormaps')
plt.show()

In [ ]:
activation_map = om.compute_activation_map(video_norm[idx - 5:idx + 20], inverted=True, interpolate=True)
om.show_activation_map(activation_map)

# Plot activation map as 3D surface
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
plt.title('Activation Map 3D Surface')
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Activation Time [ms]')
ax.set_title('Activation Map 3D Surface')
# Create a meshgrid for the x and y coordinates
x = np.arange(activation_map.shape[1])
y = np.arange(activation_map.shape[0])
X, Y = np.meshgrid(x, y)
# Flatten the activation map for plotting
Z = activation_map
# Create a surface plot
ax.plot_surface(X, Y, Z, cmap='turbo', edgecolor='none', linewidth=0, shade=False)
# Set the view angle
# ax.view_init(elev=30, azim=210)
ax.view_init(elev=40, azim=-20)
plt.tight_layout()
plt.show()

In [ ]:
import pyvista as pv

plotter = pv.Plotter(notebook=True)
plotter.set_background('white')

x = np.arange(activation_map.shape[1]).astype(np.float32)
y = np.arange(activation_map.shape[0]).astype(np.float32)
X, Y = np.meshgrid(x, y)
Z = activation_map.astype(np.float32)
grid = pv.StructuredGrid(X, Y, Z)
grid['Local Activation Time'] = Z.ravel()
# grid.plot(cmap='turbo')
plotter.add_mesh(grid, cmap='turbo')
plotter.show()

## Computing Activation Maps from Frame-Wise Difference Optical Maps

In [Tutorial 2](signal_extraction.ipynb), we introduced the frame-wise difference method to emphasize sudden temporal changes in a video. Sudden temporal changes are caused by upstrokes of the action potential or calcium transients and the frame-wise difference filter is therefore ideally suited to visualize wavefronts as they propagate across the tissue.

In [ ]:
video_diff = om.video.temporal_difference(video_filtered, 5)
video_diff[:, mask] = np.nan
video_diff_norm = om.video.normalize_pixelwise(video_diff)

The frame-wise difference approach enhances action potential upstroke, see the following video with temporal difference in the middle and our previous pixel-wise normalized video on the right:

In [ ]:
om.show_videos([video, video_diff_norm, video_norm],
               titles=["original", "frame-wise diff", "pixelwise normalized"],
               interval=100)

In [ ]:
render(lambda: om.show_videos([video, video_diff_norm, video_norm],
               titles=["original", "frame-wise diff", "pixelwise normalized"],
               interval=250))

Let's visualize the wavefront as an overlay over the raw (motion-stabilized) video. We will need to further post-process the data as follows: 

In [ ]:
video_diff[video_diff > 0] = 0
video_diff_norm = om.video.normalize_pixelwise(-video_diff)

The action potential upstroke overlaid onto the raw video:

In [ ]:
om.video.show_video_overlay(video,
                            overlay=video_diff_norm,
                            vmin_overlay=-1,
                            vmax_overlay=1)

In [ ]:
render(lambda: om.video.show_video_overlay(video, video_diff_norm, vmin_overlay=-1, vmax_overlay=1, interval=200))